<h1>CNN AutoEncoder</h1>

In this project we will build a cnn autoencoder from scratch with pytorch to reconstruct images.
the dataset will be the MNIST dataset 

In [2]:
import torch 
import torch.nn as nn
import torch.optim as optim 
from torchvision import datasets , transforms 
import matplotlib.pyplot as plt 

In [3]:
transform = transforms.ToTensor()
mnist_data = datasets.MNIST(root= "./data" , train = True, download = False, transform =transform)
data_loader = torch.utils.data.DataLoader(dataset = mnist_data, batch_size = 64, shuffle = True)

In [28]:
# # Define the sizes for training, validation, and testing
# train_size = int(0.8 * len(mnist_data))  # 80% for training
# val_size = int(0.1 * len(mnist_data))    # 10% for validation
# test_size = len(mnist_data) - train_size - val_size  # 10% for testing
# # Use random_split to create subsets
# train_data, val_data, test_data = torch.utils.data.random_split(
#     mnist_data, [train_size, val_size, test_size]
# )

<h1>Model's Architecture</h1>

In [5]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28 ,128), 
            nn.ReLU(),
            nn.Linear(128 , 64), 
            nn.ReLU(), 
            nn.Linear(64, 12),
            nn.ReLU(), 
            nn.Linear(12, 3)
        ); 
        self.decoder = nn.Sequential(
            nn.Linear(3 ,12), 
            nn.ReLU(),
            nn.Linear(12 , 64), 
            nn.ReLU(), 
            nn.Linear(64, 128),
            nn.ReLU(), 
            nn.Linear(128, 28*28), 
            nn.Sigmoid()
        );  
    def forward (self , x):
        embedding = self.encoder(x)
        output = self.decoder (embedding);
        return output
        

<h1>
    Training the Model
</h1>

In [10]:
#let us set the device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
#instantiation 
model = Autoencoder()

In [13]:
#loss function 
criterion = torch.nn.MSELoss()

#optimizer 
optimizer = optim.Adam(model.parameters(), lr=0.001)

#number of epochs 
epochs = 20 
 
#training model
def train (model , data):
    model = model.to(device) 
    for epoch in range(epochs):
        print(f"started epoch {epoch+1}");
        running_loss= 0.0 # to track the loss 
        for images ,_  in data_loader: 
            images = images.view(images.size(0), -1).to(device)            
            optimizer.zero_grad()
            outputs = model (images)
            loss = criterion (outputs, images)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(data_loader):.4f}") 
    return model 

In [14]:
trained_model = train(model , data_loader)

started epoch 1
Epoch [1/20], Loss: 0.0366
started epoch 2
Epoch [2/20], Loss: 0.0357
started epoch 3
Epoch [3/20], Loss: 0.0350
started epoch 4
Epoch [4/20], Loss: 0.0345
started epoch 5
Epoch [5/20], Loss: 0.0340
started epoch 6
Epoch [6/20], Loss: 0.0337
started epoch 7
Epoch [7/20], Loss: 0.0333
started epoch 8
Epoch [8/20], Loss: 0.0331
started epoch 9
Epoch [9/20], Loss: 0.0328
started epoch 10
Epoch [10/20], Loss: 0.0326
started epoch 11
Epoch [11/20], Loss: 0.0324
started epoch 12
Epoch [12/20], Loss: 0.0322
started epoch 13
Epoch [13/20], Loss: 0.0320
started epoch 14
Epoch [14/20], Loss: 0.0318
started epoch 15
Epoch [15/20], Loss: 0.0317
started epoch 16
Epoch [16/20], Loss: 0.0315
started epoch 17
Epoch [17/20], Loss: 0.0314
started epoch 18
Epoch [18/20], Loss: 0.0313
started epoch 19
Epoch [19/20], Loss: 0.0312
started epoch 20
Epoch [20/20], Loss: 0.0311


<h1>Testing</h1>

In [16]:
#data iterator 
dataIter = iter(data_loader)
images,  labels = next(dataIter)
print(torch.min(images) , torch.max(images))

tensor(0.) tensor(1.)


In [ ]:
## the pixel values are between 0 and 1 

In [17]:
from PIL import Image 
transform_toPil= transforms.ToPILImage()
img = transform_toPil(images[0])
img

In [20]:
#squeeze (remove the batch dimension ) so that we can flatten it then pass it to the model 
pic = images[0].squeeze()
# Flatten the input tensor to shape [1, 28*28]
input_tensor_flattened = pic.view(1, -1)  # Shape: [1, 784]
input_tensor_flattened.shape

torch.Size([1, 784])

In [22]:
input_tensor_flattened = input_tensor_flattened.to(device)

In [23]:
prediction = trained_model(input_tensor_flattened)

In [24]:
pred = prediction.view(28,28)
print(pred.shape)
pred = pred.unsqueeze(0)

torch.Size([28, 28])


In [25]:
pred.shape

torch.Size([1, 28, 28])

In [26]:
pic= transform_toPil(pred)
pic